In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/vantuan_ngo/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [1]:
import importlib

import utils
importlib.reload(utils)
from utils import *

import AD
importlib.reload(AD)
from AD import *

import utils_dataset
importlib.reload(utils_dataset)
from utils_dataset import *

/home/vantuan_ngo/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
# Load the latest model
model_path = "./models/ae_latest.pth"
if os.path.exists(model_path):
    model = torch.load(model_path, map_location=device)
    model.eval()
    print(f"Model loaded from {model_path}")
else:
    print(f"Model file {model_path} not found")

/tmp/ipykernel_2280723/122243062.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


Model loaded from ./models/ae_latest.pth


In [67]:
TEST_DF_FILE_PATH = os.path.join("../datasets/FccIQ/synthetic/test_df.csv")
print("TEST_DF_FILE_PATH: ", TEST_DF_FILE_PATH)
test_df = pd.read_csv(TEST_DF_FILE_PATH)
print(f'test_df.shape: {test_df.shape}')
print(test_df.head())

TEST_DF_FILE_PATH:  ../datasets/FccIQ/synthetic/test_df.csv
test_df.shape: (14729, 7)
   SNR  SIR  MCS  FRQ  Slot  \
0   18   -1    1   -1    75   
1   22   -1    1   -1     7   
2   34   -1    2   -1    61   
3   22   -1   12   -1    13   
4   30   -1    1   -1    49   

                                           file_path ground_truth_file_path  
0  /home/vantuan_ngo/anomalib-example/FccIQ/../da...                    NaN  
1  /home/vantuan_ngo/anomalib-example/FccIQ/../da...                    NaN  
2  /home/vantuan_ngo/anomalib-example/FccIQ/../da...                    NaN  
3  /home/vantuan_ngo/anomalib-example/FccIQ/../da...                    NaN  
4  /home/vantuan_ngo/anomalib-example/FccIQ/../da...                    NaN  


In [65]:
from utils import *
from sklearn.metrics import f1_score
# Calculate accuracy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

os.makedirs("./anomaly", exist_ok=True)

SNR = 2

test_on_a_SNR = test_df[test_df["SNR"] == SNR]
SIRs = test_on_a_SNR["SIR"].unique()
SIRs.sort()
print(SIRs)

for sir in SIRs:
    print("------- sir ", sir)
    test_sir_df = test_on_a_SNR[test_on_a_SNR["SIR"] == sir]

    print(f'test_sir_df.shape: {test_sir_df.shape}')
    tests, ground_truths = load_test_dataset(test_sir_df)

    print(f'tests.shape: {tests.shape}')
    print(f'ground_truths.shape: {ground_truths.shape}')

    # print(tests.shape)
    # print(ground_truths.shape)

    test_dataset = TensorDataset(tests, ground_truths)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    anomaly_maps = []
    anomaly_bins = []
    verdicts = []
    ground_truths = []
    for batch_idx, (test_batch, ground_truth_batch) in enumerate(test_loader):
        test_batch = test_batch.to(device)
        ground_truth_batch = ground_truth_batch.to(device)
        recon, _, _ = model(test_batch)

        anomaly_map_batch = generate_anomaly_map(test_batch, recon)
        anomaly_map_batch = anomaly_map_batch.detach().cpu().numpy()
        dynamic_threshold = np.percentile(anomaly_map_batch, 99)
        threshold = max(0.041708, dynamic_threshold)
        # print(f"Threshold for frequency {frq:08d}: {threshold:.04f}")
        # print(f"Threshold for frequency {frq:08d}: {threshold:.04f}")
        anomaly_bin_batch = (anomaly_map_batch > threshold)
        # print(f"anomaly_map_batch.shape: {anomaly_map_batch.shape}")
        # print(f"anomaly_bin_batch.shape: {anomaly_bin_batch.shape}")
        ground_truth_batch = ground_truth_batch.detach().cpu().numpy()

        verdicts_batch = [[1] if np.sum(anomaly_map) > 300*14*0.05 else [0] for anomaly_map in anomaly_map_batch]

        anomaly_maps.append(anomaly_map_batch)
        anomaly_bins.append(anomaly_bin_batch)
        verdicts.append(verdicts_batch)
        ground_truths.append(ground_truth_batch)

    # Concatenate all batches
    anomaly_maps = np.concatenate(anomaly_maps, axis=0)
    anomaly_bins = np.concatenate(anomaly_bins, axis=0)
    verdicts = np.concatenate(verdicts, axis=0)
    ground_truths = np.concatenate(ground_truths, axis=0)

    anomaly_maps = np.array(anomaly_maps)
    anomaly_bins = np.array(anomaly_bins).astype(np.int32)
    verdicts = np.array(verdicts).astype(np.int32)
    ground_truths = np.array(ground_truths).astype(np.int32)
    print(f"anomaly_maps.shape: {anomaly_maps.shape}")
    print(f"anomaly_bins.shape: {anomaly_bins.shape}")
    print(f"verdicts.shape: {verdicts.shape}")
    print(f"ground_truths.shape: {ground_truths.shape}")

    # Calculate accuracy
    accuracy = accuracy_score(ground_truths, verdicts)
    precision = precision_score(ground_truths, verdicts, average='binary')
    recall = recall_score(ground_truths, verdicts, average='binary')
    f1 = f1_score(ground_truths, verdicts, average='binary')

    # Calculate confusion matrix with explicit labels to avoid warning
    cm = confusion_matrix(ground_truths, verdicts, labels=[0, 1])

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:")
    print(cm)

    gen_report(f"./anomaly/anomaly_maps_SNR_{SNR}_SIR_{sir}", anomaly_maps, anomaly_bins, verdicts, ground_truths)

[-1  2  4  6  8 10 14 18 22 26 30 34 38]
------- sir  -1
test_sir_df.shape: (71, 7)
tests.shape: torch.Size([71, 2, 300, 14])
ground_truths.shape: torch.Size([71, 1])
anomaly_maps.shape: (71, 1, 300, 14)
anomaly_bins.shape: (71, 1, 300, 14)
verdicts.shape: (71, 1)
ground_truths.shape: (71, 1)
Accuracy: 1.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Confusion Matrix:
[[71  0]
 [ 0  0]]
Accuracy: 1.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Confusion Matrix:
[[71  0]
 [ 0  0]]


/home/vantuan_ngo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vantuan_ngo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vantuan_ngo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vantuan_ngo/.local/lib/pyt

Building PDF
PDF built
------- sir  2
test_sir_df.shape: (745, 7)
tests.shape: torch.Size([745, 2, 300, 14])
ground_truths.shape: torch.Size([745, 1])
anomaly_maps.shape: (745, 1, 300, 14)
anomaly_bins.shape: (745, 1, 300, 14)
verdicts.shape: (745, 1)
ground_truths.shape: (745, 1)
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Confusion Matrix:
[[  0   0]
 [745   0]]
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Confusion Matrix:
[[  0   0]
 [745   0]]


/home/vantuan_ngo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vantuan_ngo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Building PDF
PDF built
------- sir  4
test_sir_df.shape: (783, 7)
tests.shape: torch.Size([783, 2, 300, 14])
ground_truths.shape: torch.Size([783, 1])
anomaly_maps.shape: (783, 1, 300, 14)
anomaly_bins.shape: (783, 1, 300, 14)
verdicts.shape: (783, 1)
ground_truths.shape: (783, 1)
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Confusion Matrix:
[[  0   0]
 [783   0]]
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Confusion Matrix:
[[  0   0]
 [783   0]]


...

Building PDF
PDF built
------- sir  34
test_sir_df.shape: (1275, 7)
tests.shape: torch.Size([1275, 2, 300, 14])
ground_truths.shape: torch.Size([1275, 1])
anomaly_maps.shape: (1275, 1, 300, 14)
anomaly_bins.shape: (1275, 1, 300, 14)
verdicts.shape: (1275, 1)
ground_truths.shape: (1275, 1)
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Confusion Matrix:
[[   0    0]
 [1275    0]]
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Confusion Matrix:
[[   0    0]
 [1275    0]]


/home/vantuan_ngo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vantuan_ngo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Building PDF
PDF built
------- sir  38
test_sir_df.shape: (980, 7)
tests.shape: torch.Size([980, 2, 300, 14])
ground_truths.shape: torch.Size([980, 1])
anomaly_maps.shape: (980, 1, 300, 14)
anomaly_bins.shape: (980, 1, 300, 14)
verdicts.shape: (980, 1)
ground_truths.shape: (980, 1)
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Confusion Matrix:
[[  0   0]
 [980   0]]
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Confusion Matrix:
[[  0   0]
 [980   0]]


/home/vantuan_ngo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vantuan_ngo/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Building PDF
PDF built
